<a href="https://colab.research.google.com/github/OmarMoghazy/Eigenfaces/blob/main/Bayes_Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [172]:
import numpy as np
import pandas as pd

from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import CategoricalNB

import itertools

# Question 2

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
label = []
for i in range(1,41):
    for j in range(1,11):
        img = Image.open("/content/gdrive/My Drive/at&t db of faces/s"+str(i) +"/"+ str(j) + '.pgm')
        numpydata = np.asarray(img).flatten()
        if i == 1 and j == 1:
            data = numpydata
        else:
            data = np.vstack((data, numpydata))
        label = np.hstack((label, i))

In [4]:
# training, testing, training_labels, testing_labels = train_test_split(
#  data, label, test_size=0.1, random_state=42)

## Split Data Into Testing Det and Training Set

In [5]:
training = data[1::2]
testing = data[::2]

training_labels = label[1::2]
testing_labels = label[::2]

## Naive Bayes Implementation 

In [6]:
def naive_bayes_train(data, labels):

  if data.shape[0] < 1:
    return
  
  class_means = np.mean(data[labels == 1], axis = 0)
  priors = data[labels==1].shape[0] / data.shape[0]
  centered_class = data[labels == 1] - class_means
  attribute_vars = np.var(centered_class, axis=0)
  # attribute_var = 1/10 * (centered_class.T @ centered_class)

  for i in range(1,np.unique(labels).size):

    prior = data[labels==i+1].shape[0] / data.shape[0]
    
    class_mean = np.mean(data[labels==i+1], axis=0)
    
    centered_class = data[labels==i+1] - class_mean
    attribute_var = np.var(centered_class, axis=0)
    # attribute_var = 1/10 * (centered_class.T @ centered_class)

    priors = np.hstack( (priors, prior) )
    class_means = np.vstack( (class_means, class_mean) )
    attribute_vars = np.vstack( (attribute_vars, attribute_var) )

  
  return priors, class_means, attribute_vars

In [96]:
def naive_bayes_classify(x, priors, class_means, attribute_vars):
  k = priors.shape[0]
  d = class_means.shape[1]
  # print(max)
  # print(k, d)
  res = []
  for sample in range(x.shape[0]):
    max = float('-inf')
    max_ind = -1
    for i in range(k):
      y = 1
      for j in range(d):
        a = 1 / np.sqrt(2 * np.pi * attribute_vars[i,j])
        b = np.exp(-1* (x[sample,j] - class_means[i,j])**2 / (2 * attribute_vars[i,j]))
        y *=  a*b
      y *=  1 / priors[i]
      if y > max:
        max = y
        max_ind = i+1
    res.append(max_ind)
  return res
      

## Classification Using Naive Bayes

In [65]:
p, c, v = naive_bayes_train(training, training_labels)
naive_bayes_classify(testing[50:60], p, c, v)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

## Dimensionality Reduction Using PCA, Keeping Only 40 Components.

In [11]:
training_mean = np.mean(training, axis=0)
training_centered_data = training - training_mean

scaler = StandardScaler()
scaler.fit(training_centered_data)
training_centered_data = scaler.transform(training_centered_data)

covariance = np.cov(training_centered_data.T, bias=True)
eigenvalues, eigenvectors = np.linalg.eigh(covariance)
idx = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

eigenvectors = eigenvectors[:, :40]

In [12]:
projected_training_data = training_centered_data @ eigenvectors

In [25]:
testing_mean = np.mean(testing, axis=0)
testing_centered_data = testing - testing_mean

scaler = StandardScaler()
scaler.fit(testing_centered_data)
testing_centered_data = scaler.transform(testing_centered_data)

In [26]:
projected_testing_data = testing_centered_data @ eigenvectors

## Classification Using Naive Bayes Again Using the New Data With Reduced Dimensions.

In [56]:
p,c,v = naive_bayes_train(projected_training_data, training_labels)
results = naive_bayes_classify(projected_testing_data, p,c,v)

In [63]:
df = pd.DataFrame(confusion_matrix(results, testing_labels))
accuracy = np.trace(df) / np.sum(np.sum(df))

In [71]:
print("Accuracy:", str(accuracy*100) + "%")

Accuracy: 88.0%


I could only get results after dimensionality reduction. I believe 10304 multiplications of very small numbers makes all the probabilities become 0 at the end, so classification fails.

The following is a confusion matrix of the results. The count on the diagonal of the matrix is the number of samples correctly classified in this class. Anything off the diagonal means there was a misclassification. For example, a sample in class 1 (0 in the matrix because of the pandas index) was incorrectly classified as class 17. 

In [72]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2
1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Question 3

In [146]:
points = np.array([
               [2,6],
               [3,5],
               [4,4],
               [5,3],
               [6,2],
               [6,4],
               [6,6],
               [8,4],
               [9,2],
               [9,3],
               [3,3],
               [4,3],
               [4,5],
               [5,5],
               [7,3],
               [7,4],
               [7,5],
               [7,2],
               [10,1],
               [10,3],
               [10,5],
               [11,3],
               [11,4],
               [12,2],
               [13,4.5]
])

labels = np.array([1]*10 +[2]*7 + [3]*8)

to_be_classified = np.array([
                             [6,5],
                             [9,4],
                             [8,5]
])

## Full Bayes Implementation

In [144]:
def full_bayes_train(data, labels):
  
  if data.shape[0] < 1:
    return
  
  class_means = np.mean(data[labels == 1], axis = 0)
  priors = data[labels==1].shape[0] / data.shape[0]
  centered_class = data[labels == 1] - class_means
  covars = [np.cov(centered_class.T, bias=True)]

  for i in range(1,np.unique(labels).size):

    prior = data[labels==i+1].shape[0] / data.shape[0]
    
    class_mean = np.mean(data[labels==i+1], axis=0)
    
    centered_class = data[labels==i+1] - class_mean
    covar = np.cov(centered_class.T, bias=True)

    priors = np.hstack( (priors, prior) )
    class_means = np.vstack( (class_means, class_mean) )
    covars.append(covar)

  covars = np.array(covars)
  
  return priors, class_means, covars

In [143]:
def full_bayes_classify(x, priors, class_means, vars):
  k = priors.shape[0]
  d = class_means.shape[1]
  res = []
  for sample in range(x.shape[0]):
    max = float('-inf')
    max_ind = -1
    for i in range(k):
      y = 1 / (np.sqrt(2 * np.pi) ** d) * np.exp(-1*((x[sample] - class_means[i]).T @ np.linalg.inv(vars[i]) @ (x[sample] - class_means[i]))/2)
      if y > max:
        max = y
        max_ind = i+1
    res.append(max_ind)
  return res

## Classification Using Full Bayes

In [142]:
p,c,v = full_bayes_train(points, labels)
results = full_bayes_classify(to_be_classified,p,c,v)
print(results)

[1, 3, 2]


## Classification using Naive Bayes

In [141]:
p,c,v = naive_bayes_train(points, labels)
results = naive_bayes_classify(to_be_classified,p,c,v)
print(results)

[2, 3, 1]


In [175]:
new_features = np.array([
               [0, 1, 1],
               [0, 1, 1],
               [0, 1, 1],
               [0, 1, 1],
               [0, 1, 1],
               [0, 1, 1],
               [0, 0, 1],
               [0, 0, 1],
               [0, 0, 1],
               [0, 0, 1],
               [0, 1, 1],
               [0, 1, 1],
               [0, 0, 1],
               [0, 0, 1],
               [0, 0, 1],
               [0, 0, 1],
               [0, 0, 1],
               [0, 0, 1],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
])

In [163]:
def categorize_x1(x):
  if x<5:
    return 0
  elif x<10:
    return 1
  else:
    return 2

def categorize_x2(x):
  if x<4:
    return 0
  else:
    return 1

In [182]:
categorized_x1 = np.array([categorize_x1(x) for x in points[:,0]])
categorized_x2 = np.array([categorize_x2(x) for x in points[:,1]])
new_points = np.hstack((np.atleast_2d(categorized_x1).T, np.atleast_2d(categorized_x2).T, new_features))

to_be_classified_new_features = np.array([
                             [0,0,1],
                             [0,0,1],
                             [1,1,0]
])

categorized_x1 = np.array([categorize_x1(x) for x in to_be_classified[:,0]])
categorized_x2 = np.array([categorize_x2(x) for x in to_be_classified[:,1]])
new_to_be_classified = np.hstack((np.atleast_2d(categorized_x1).T, np.atleast_2d(categorized_x2).T, to_be_classified_new_features))

In [165]:
print(new_to_be_classified)

[[1 1 0 0 1]
 [1 1 0 0 1]
 [1 1 1 1 0]]


I believe the following error is because of the absence of pseudo counting in the sklearn categorical NB implementation. I did not have time to implement it on my own, but I intend to do it as soon as I can.

In [183]:
clf = CategoricalNB()
clf.fit(new_points, labels)
CategoricalNB()
print(np.shape(new_to_be_classified))
clf.predict(new_to_be_classified)

(3, 5)


IndexError: ignored